Iceberg Survival Guide: Now, you can see the Iceberg icon next to managed tables. From now on, you can create a managed Delta or iceberg table. I think that soon, the formats will unify, but to avoid headaches for now, you need to know a few things:

- no CDC in Iceberg,
- we need to know the manifest concept 
- liquid partitioning is supported but needs some additional table properties
- Maintenance: OPTIMIZE or VACUUM in UC is made by the same commands as for Delta.

In [0]:
CREATE OR REPLACE TABLE hub.default.iceberg_orders (
  order_id     BIGINT,
  customer_id  BIGINT,
  order_ts     TIMESTAMP,
  total_amt    DECIMAL(12,2)
)
USING ICEBERG;

INSERT INTO hub.default.iceberg_orders (order_id, customer_id, order_ts, total_amt)
VALUES (1, 12345, '2025-06-29 00:00:00', 100.00);

In [0]:
CREATE OR REPLACE TABLE hub.default.ice_orders (
  order_id     BIGINT,
  customer_id  BIGINT,
  order_ts     TIMESTAMP,
  total_amt    DECIMAL(12,2)
)
USING ICEBERG
CLUSTER BY (customer_id)
TBLPROPERTIES (
  'write.target-file-size-bytes' = '536870912', -- Target file size for written files in bytes (512 MB)
  'write.manifest.target-file-size-bytes' = '16777216', -- Target file size for manifest files in bytes (16 MB)
  'history.expire.min-snapshots-to-keep' = '1', -- Minimum number of snapshots to keep when expiring history
  'commit.retry.num-retries' = '3', -- Number of retries for commit operations
  'commit.retry.total-timeout-ms' = '60000', -- Total timeout for commit retries in milliseconds (60 seconds)
  'read.split.target-size' = '134217728', -- Target size for read splits in bytes (128 MB)
  'read.split.open-file-cost' = '4194304', -- Cost of opening a file for read splits in bytes (4 MB)
  'write.sort.order' = 'order_id,customer_id', -- Sort order for written data
  'metadata.previous-versions-max' = '5', -- Maximum number of previous metadata versions to keep
  'object.tagging.enabled' = 'false', -- Enable or disable object tagging
  'write.delete-vector.enabled' = 'false', -- Disable deletion vectors
  'write.row-id.enabled' = 'false' -- Disable row IDs
);

-- 3️⃣  Insert the first two rows
INSERT INTO ice_orders VALUES
  (1, 101, '2025-06-26T10:00:00',  99.99),
  (2, 102, '2025-06-26T10:05:00', 149.50);

-- 4️⃣  Append one more row later on
INSERT INTO ice_orders VALUES
  (3, 103, '2025-06-26T10:10:00', 200.00);

In [0]:
OPTIMIZE hub.default.iceberg_orders

In [0]:
VACUUM hub.default.iceberg_orders